In this notebook I will be using Iris dataset and will apply PCA on that. I will import iris dataset from UCI repository and will read it using pandas. We can also import it using sklearn inbuilt module datasets. 

In [1]:
import pandas as pd

df = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
    header=None, 
    sep=',')#importing from UCI repository

df.columns=['sepal_lenth(cm)', 'sepal_width(cm)', 'petal_length(cm)', 'petal_width(cm)', 'class_name']#assigning column names.
#df.dropna(how="all", inplace=True) # drops the empty line at file-end

df.tail()

,sepal_lenth(cm),sepal_width(cm),petal_length(cm),petal_width(cm),class_name
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [2]:
#splitting dataset into data X and labels y
X = df.iloc[:,0:4].values
y = df.iloc[:,4].values

Standadizing the data i.e converting it onto a unit scale(mean=0, variance=1)

In [3]:
from sklearn.preprocessing import StandardScaler
X_std=StandardScaler().fit_transform(X)

Compute co-variance matrix and perform eigen decomposition to obtain eigen vectors, eigen values.(We can even do Singular Vector Decomposition and other methods too) 

In [4]:
#Computing covariance matrix
import numpy as np
print('NumPy covariance matrix: \n%s' %np.cov(X_std.T))

NumPy covariance matrix: 
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]


In [5]:
#Performing Eigen decomposition on Covariance matrix to get eigen vectors and eigen values
Cov_matrix=np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(Cov_matrix)

print('Eigen_vectors \n%s' %eig_vecs)
print('\nEigen_values \n%s' %eig_vals)

Eigen_vectors 
[[ 0.52237162 -0.37231836 -0.72101681  0.26199559]
 [-0.26335492 -0.92555649  0.24203288 -0.12413481]
 [ 0.58125401 -0.02109478  0.14089226 -0.80115427]
 [ 0.56561105 -0.06541577  0.6338014   0.52354627]]

Eigen_values 
[2.93035378 0.92740362 0.14834223 0.02074601]


In [6]:
#Selection of Eigen vectors(Doing PCA simply obtains new axes which are Eigen vectors)
# Make a list of (eigenvalue, eigenvector)
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
#print(type(eig_pairs))
# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()
#Lets check whether they are arranged in descencing order or not
for i in range(len(eig_pairs)):
    print(eig_pairs[i])

(2.9303537755893174, array([ 0.52237162, -0.26335492,  0.58125401,  0.56561105]))
(0.9274036215173421, array([-0.37231836, -0.92555649, -0.02109478, -0.06541577]))
(0.14834222648163944, array([-0.72101681,  0.24203288,  0.14089226,  0.6338014 ]))
(0.020746013995595943, array([ 0.26199559, -0.12413481, -0.80115427,  0.52354627]))


In [7]:
#We perform "Variance explained" to know which principal components contribute to most variance which is selection of PCA
import warnings
warnings.filterwarnings('ignore')
import plotly.plotly as py
import plotly.graph_objs as st
#plotly.tools.set_credentials_file(username='Manideep07', api_key='0zeiCRCcVSxTimk50q6F')
#py.sign_in("Manideep07","0zeiCRCcVSxTimk50q6F")
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = st.Bar(
        x=['PC %s' %i for i in range(1,5)],
        y=var_exp,
        showlegend=False)

trace2 = st.Scatter(
        x=['PC %s' %i for i in range(1,5)], 
        y=cum_var_exp,
        name='cumulative variance')

data_1 =st.Data([trace1, trace2]) 

layout=st.Layout(
        yaxis=st.layout.YAxis(title='variance explained in percent'),
        title='Explained variance by different principal components')

fig = st.Figure(data=data_1, layout=layout)
py.iplot(fig)

C:\Users\mani\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




From the above visualization we can see that PC1 accounts for 72.77% of variance, PC2 accounts for 22.03,.. and it decreases as we go on further with other principal components 

Now comes the final part, the generation of projection matrix. We use this matrix to transform the iris dataset from 4 feature space to 2 feature space.(Number of principal components) 

In [8]:
#Construction of projection matrix
matrix_w = np.hstack((eig_pairs[0][1].reshape(4,1), 
                      eig_pairs[1][1].reshape(4,1)))

print('Matrix W:\n', matrix_w)

Matrix W:
 [[ 0.52237162 -0.37231836]
 [-0.26335492 -0.92555649]
 [ 0.58125401 -0.02109478]
 [ 0.56561105 -0.06541577]]


Projecting it into new feature space where we finally have 150 samples with 2 features. We transform it with the equation X_new=X x Y, where X is original data matrix and Y is projection matrix. 

In [9]:
X_new=X_std.dot(matrix_w)

Visualizing the 150 samples distribution around 2 principal components 

In [10]:
traces = []

import plotly.graph_objs as sl

for name in ('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'):

    trace = st.Scatter(
        x=X_new[y==name,0],
        y=X_new[y==name,1],
        mode='markers',
        name=name,
        marker=st.scatter.Marker(
            size=12,
            line=st.Line(
                color='rgba(215, 215, 215, 0.14)',
                width=0.5),
            opacity=0.8),visible=True)
    traces.append(trace)

data =traces
layout = st.Layout(xaxis=dict(title='PC1', showline=False),
                yaxis=dict(title='PC2', showline=False))
fig = dict(data=data, layout=layout)
py.iplot(fig,filename='scatter-mode')

C:\Users\mani\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:372: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




To avoid all the tedious work, we have this inbuilt in scikit learn package.  

In [11]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(X_std) #This does the complete job for us instead of writing all the code.

Visualizing the 150 samples distribution around 2 principal components(for scikit learn one)

In [12]:
import warnings
warnings.filterwarnings('ignore')
traces = []

for name in ('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'):

    trace = st.Scatter(
        x=Y_sklearn[y==name,0],
        y=Y_sklearn[y==name,1],
        mode='markers',
        name=name,marker=dict(
            size=12,
            line=st.Line(
                color='rgba(215, 215, 215, 0.14)',
                width=0.5),
            opacity=0.8))
    traces.append(trace)


data = traces
layout = st.Layout(xaxis=st.XAxis(title='PC1', showline=False),
                yaxis=st.YAxis(title='PC2', showline=False))
fig = st.Figure(data=data, layout=layout)
py.iplot(fig)